## KB 시세

In [4]:
#[예제 2.2] xlwings모듈 활용하여 엑셀 데이터 가져와서 데이터프레임으로 저장 

#import pandas as pd

import xlwings as xw

path = r'데이터\KBLiivon_2019129.xls'
wb = xw.Book(path) 

sheet = wb.sheets['KBLiivon_2019129']   
row_num = sheet.range(1,1).end('down').end('down').row  
data_range = 'A3:B' + str(row_num)
#apt_info_column = ["단지명", "주소(지번)", "주소(도로명)", "공급/전용면적", "세대수", "방/욕실수", "현관구조"]
apt_info = sheet[data_range].options(pd.DataFrame, index=False, header=False).value 

### 단지 정보

In [5]:
import re
apt_info_column = apt_info[0]
apt_info_row = apt_info[1]
apt_info_column = apt_info_column.tolist()


def makeAptInfo(apt_info_row, apt_info_column):
    apt_info_row = apt_info_row.tolist()    
    apt = pd.DataFrame([apt_info_row], columns=apt_info_column)
    areas = apt["공급/전용면적"][0].split('/')
    apt["공급"] = float(areas[0])
    apt["전용"] = float(areas[1].replace("㎡", ""))
    households = apt["세대수"][0].split(' ')
    
    household = int(re.findall('\d+', households[0])[0])
    totalHousehold = int(re.findall('\d+', households[1])[0])
    tRooms = apt["방/욕실수"][0].split('/')
    rooms = int(re.findall('\d+', tRooms[0])[0])
    bathroom = int(re.findall('\d+', tRooms[1])[0])
    apt["방수"] = rooms
    apt["욕실수"] = bathroom
    apt = apt.drop(["세대수", "공급/전용면적", "방/욕실수"], axis=1)
    apt["세대수"] = household
    apt["총세대수"] = totalHousehold
    return apt

In [6]:
apt = makeAptInfo(apt_info_row, apt_info_column)

### 시세 갱신일

In [7]:

def makeRefreshingDate(sheet, df):    
    price_refreshing_row_num = sheet.range(11,1).row
    price_refreshing_data_range = 'A11:A' + str(price_refreshing_row_num)
    price_refreshing_info = sheet[price_refreshing_data_range].options(pd.DataFrame, index=False, header=False).value
    price_refreshing_info = price_refreshing_info[0][0].split(' ')
    
    from datetime import datetime
    print(price_refreshing_info[3])
    price_refreshing_info = datetime.strptime(price_refreshing_info[3], '%Y.%m.%d')
    
    df.insert(loc=0, column="시세갱신일", value=price_refreshing_info)
    df.reset_index(drop=True)
    df.index += 1

    return df

In [8]:
makeRefreshingDate(sheet, apt)

2019.12.06


,시세갱신일,단지명,주소(지번),주소(도로명),현관구조,공급,전용,방수,욕실수,세대수,총세대수
1,2019-12-06,경동윈츠리버,서울특별시 동작구 본동 484,서울특별시 동작구 노량진로32길,계단식,78.93,56.68,2,1,142,272


### 시세정보

In [9]:
def reorganizeColumns(df, strList, baseIndex):
    bignames = makeColumns(strList)
    big_col = list(df.columns)
    small_col = list(df.iloc[baseIndex])
    
    setColumns(big_col, small_col, bignames)    
    df.columns = [big_col, small_col]

In [10]:
def setColumns(big_col, small_col, bignames):
    for num, col in enumerate(small_col):
        #print(num, col)
        if col == None:
            small_col[num] = big_col[num]

        check = num
        while True:
            if big_col[check] in bignames:                
                big_col[num] = big_col[check]
                #print('big_col[', check, ']:', big_col[check], 'big_col[', num,']:', big_col[num])
                break
            else:
                #print('ELSE::',check)
                check = check - 1

In [ ]:
def makeColumns(list):
    colList = list.split(' ')
    return colList

In [ ]:
price_row_num = sheet.range(14,1).end('down').row  
price_data_range = 'A12:I' + str(price_row_num)
apt_price_info = sheet[price_data_range].options(pd.DataFrame, index=False, header=True).value 
apt_price_info

In [ ]:
reorganizeColumns(apt_price_info, '기준월 매매가 전세가 월세가', 0)

In [ ]:
apt_price_info

In [ ]:
apt_price_info.columns

In [ ]:
new_price_col_data = apt_price_info.drop([0, 0])

In [ ]:
new_price_col_data = new_price_col_data.sort_values(by=new_price_col_data.columns[0], ascending=True)

In [ ]:
new_price_col_data = new_price_col_data.reset_index(drop=True)
new_price_col_data.index += 1

In [ ]:
from datetime import datetime

new_price_col_data['기준월'] = pd.to_datetime(new_price_col_data['기준월']['기준월'], errors='coerce')
new_price_col_data

In [ ]:
# new_price_col_data['전세가', '하위평균가'] = new_price_col_data['전세가', '하위평균가'].replace('-', int(0))
# new_price_col_data['전세가', '일반평균가'] = new_price_col_data['전세가', '일반평균가'].replace('-', int(0))
# new_price_col_data['전세가', '일반평균가'] = new_price_col_data['전세가', '일반평균가'].replace('-', int(0))
new_price_col_data['전세가'] = new_price_col_data['전세가'].replace('-', 0)
new_price_col_data

### 전세가/매매가 차트

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
 

In [ ]:
from matplotlib import font_manager, rc
import matplotlib.pylab as plt
%matplotlib inline

plt.rcParams["figure.figsize"] = (14,6)
plt.rcParams['lines.linewidth'] = 2
plt.rcParams['lines.color'] = 'r'
plt.rcParams['axes.grid'] = True 
font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
rc('font', family=font_name)

In [ ]:
ax = plt.gca()
new_price_col_data.plot(kind='line', x=new_price_col_data.columns[0], y=new_price_col_data.columns[3], ax=ax)
new_price_col_data.plot(kind='line', x=new_price_col_data.columns[0], y=new_price_col_data.columns[5], ax=ax)
plt.show()

### 가치 차트

In [ ]:
import xlwings as xw
import pandas as pd

path = r'{}'
wb_value = xw.Book(path) 
print(path)
value_sheet = wb_value.sheets['sheet1']   
row_num = value_sheet.range(1,1).end('down').end('down').row  
data_range = 'A1:F' + str(row_num)
# #apt_info_column = ["단지명", "주소(지번)", "주소(도로명)", "공급/전용면적", "세대수", "방/욕실수", "현관구조"]
value_info = value_sheet[data_range].options(pd.DataFrame, index=False, header=True).value 
value_info = value_info.drop([0, 0])

In [ ]:
value_info["년도"] = pd.to_datetime(value_info["년도"], format='%Y', errors='ignore')

In [ ]:

value_info["년도"] = pd.DatetimeIndex(value_info["년도"]).year
value_info["도시근로자 평균월소득(A)"] = value_info["도시근로자 평균월소득(A)"].map('{:,.0f}'.format)
value_info["평균연소득 (A X 12개월)"] = value_info["평균연소득 (A X 12개월)"].map('{:,.0f}'.format)
value_info["KB시세 및 매수가능가격"] = 0
#value_info["매매가격"] = value_info["매매가격"].map('{:,.0f}'.format)
value_info["매매가격"] = 0
#value_info["가치"] = value_info["가치"].map('{:,.2f}'.format)
value_info["가치"] = 0


In [ ]:
t_value_price = new_price_col_data[new_price_col_data["기준월"]["기준월"].dt.month == 10]
t_value_price = t_value_price.reset_index(drop=True)
t_value_price['기준월'] = t_value_price['기준월']['기준월'].dt.year
t_value_price = t_value_price.reset_index(drop=True)
t_value_price.index += 1
value_price_list = t_value_price["기준월"]["기준월"].tolist()

In [ ]:
for idx, year in enumerate(value_price_list):   
    
    tValue = t_value_price.loc[t_value_price['기준월']['기준월'] == year, '매매가']['상위평균가'].values[0]
    value_info.loc[value_info['년도'] == year, 'KB시세 및 매수가능가격'] = tValue
            

        
value_info["도시근로자 평균월소득(A)"] = value_info["도시근로자 평균월소득(A)"].str.replace(",", "")
value_info["평균연소득 (A X 12개월)"] = value_info["평균연소득 (A X 12개월)"].str.replace(",", "")

value_info["도시근로자 평균월소득(A)"] = pd.to_numeric(value_info["도시근로자 평균월소득(A)"])
value_info["평균연소득 (A X 12개월)"] = pd.to_numeric(value_info["평균연소득 (A X 12개월)"])
#value_info["년도"] = pd.to_datetime(value_info["년도"].astype(str), format='%Y')
dropIndexs = value_info[value_info['KB시세 및 매수가능가격'] == 0].index
value_info.drop(dropIndexs, inplace=True)
value_info.reset_index(drop=True)

value_info.index = np.arange(1, len(value_info) + 1)

value_info["매매가격"] = value_info["KB시세 및 매수가능가격"] * 10000

value_info["가치"] = value_info["매매가격"] / value_info["평균연소득 (A X 12개월)"]
value_info["평균가치"] = value_info["가치"].sum() / value_info["가치"].count()
#value_info["평균가치"] = value_

#value_info = value_info.set_index('년도')
value_info

In [ ]:
from matplotlib import font_manager, rc
import matplotlib.pylab as plt
from matplotlib import pyplot
import numpy as np
import string

%matplotlib inline

plt.rcParams["figure.figsize"] = (14,6)
plt.rcParams['lines.linewidth'] = 3
plt.rcParams['lines.color'] = 'r'
plt.rcParams['axes.grid'] = True 

font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
rc('font', family=font_name)


In [ ]:
ax_value_info = plt.gca()
totalCount = value_info["가치"].count()
for index, value in value_info.iterrows():    
    
    x = int(value[0])
    y = value[5]
    comment = str("{0:.2f}".format(value[5]))

    ax_value_info.text(x, y, comment, fontsize=15)
    
    if totalCount == index:
        ax_value_info.text(x, value[6], '{:,.2f}'.format(value[6]), fontsize=20)


value_info.plot(kind='line', x=value_info.columns[0], y=value_info.columns[5], ax=ax_value_info)
value_info.plot(kind='line', x=value_info.columns[0], y=value_info.columns[6], ax=ax_value_info)

plt.show()

### 심리차트

In [ ]:
import pandas as pd
import xlwings as xw

path = r'{}'
wb = xw.Book(path) 
print(path)
sheet = wb.sheets['{}']   
row_num = sheet.range(1,1).end('down').end('down').row  
data_range = 'B2:BV' + str(row_num)
#apt_info_column = ["단지명", "주소(지번)", "주소(도로명)", "공급/전용면적", "세대수", "방/욕실수", "현관구조"]
mentality = sheet[data_range].options(pd.DataFrame, index=False, header=True).value 

In [ ]:
reorganizeColumns(mentality, '날짜 전국 서울 강북 강남 6개광역시 5개광역시 부산 대구 인천 광주 대전 울산 세종 수도권 경기도 강원도 충청북도 충청남도 전라북도 전라남도 경상북도 경상남도 기타지방 제주', 0)
mentality = mentality.drop([0,0])
mentality.fillna(0)

In [ ]:
from matplotlib import font_manager, rc
import matplotlib.pylab as plt
from matplotlib import pyplot
import numpy as np
import string

%matplotlib inline

plt.rcParams["figure.figsize"] = (14,6)
plt.rcParams['lines.linewidth'] = 3
plt.rcParams['lines.color'] = 'r'
plt.rcParams['axes.grid'] = True 

font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
rc('font', family=font_name)

In [ ]:
ax_mentality = plt.gca()
# totalCount = mentality["가치"].count()
# for index, value in value_info.iterrows():    
    
#     x = int(value[0])
#     y = value[5]
#     comment = str("{0:.2f}".format(value[5]))

#     ax_value_info.text(x, y, comment, fontsize=15)
    
#     if totalCount == index:
#         ax_value_info.text(x, value[6], '{:,.2f}'.format(value[6]), fontsize=20)

x = mentality.columns[0]
y = [40, 41]
mentality.plot(kind='line', x=x, y=y, ax=ax_mentality)
#value_info.plot(kind='line', x=value_info.columns[0], y=value_info.columns[6], ax=ax_value_info)

plt.show()

In [ ]:
from datetime import datetime
new_mentality = mentality
ret = pd.to_datetime(mentality['날짜']['날짜'], errors='coerce')
mentality['날짜'] = ret
new_mentality = mentality[mentality["날짜"]["날짜"].dt.year >= 2018]

In [ ]:
ax_mentality = plt.gca()
# totalCount = mentality["가치"].count()
# for index, value in value_info.iterrows():    
    
#     x = int(value[0])
#     y = value[5]
#     comment = str("{0:.2f}".format(value[5]))

#     ax_value_info.text(x, y, comment, fontsize=15)
    
#     if totalCount == index:
#         ax_value_info.text(x, value[6], '{:,.2f}'.format(value[6]), fontsize=20)

x = new_mentality.columns[0]
y = [4, 5]
new_mentality.plot(kind='line', x=x, y=y, ax=ax_mentality)
#value_info.plot(kind='line', x=value_info.columns[0], y=value_info.columns[6], ax=ax_value_info)

plt.show()